In [ ]:
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), '../core'))
import kaggle_support as kgs
import importlib
import matplotlib.pyplot as plt
import numpy as np
import cupy as cp
from dataclasses import dataclass, field, fields
import pack_cuda
import pack_vis
import pack_cost
import copy
import time
import pack_ga
from IPython.display import HTML, display, clear_output
pack_cuda.USE_FLOAT32 = True
pack_cuda._ensure_initialized()


In [ ]:
import pack_runner
fastMode = False
runner = pack_runner.baseline_runner_fixed_h(fast_mode=False)

runner.base_ga.N_trees_to_do = np.array([40])
runner.base_ga.plot_fitness_predictors = False
runner.base_ga.plot_diversity_matrix = True
runner.base_ga.plot_champions = True

kgs.profiling=False
runner.run()

In [ ]:
runner.result_ga.best_cost_per_generation

In [ ]:
kgs.dill_save(kgs.code_dir+'../../results/test_ga_result_ga.dill',runner.result_ga)